## ETL DIM_STORE

In [65]:
import pandas as pd
from config.db_config import source, conn

### CAR

In [66]:
df_store = pd.read_sql_query('select * from store', source)
df_address = pd.read_sql_query("select * from address", source)
df_city = pd.read_sql_query("select * from city", source)
df_country = pd.read_sql_query("select * from country", source)

### TRA

In [67]:
merged_df = pd.merge(df_store, df_address, how='inner', on='address_id')
merged_df = pd.merge(merged_df, df_city, how='inner', on='city_id')
merged_df = pd.merge(merged_df, df_country, how='inner', on='country_id', suffixes=('merged', '_last_update_x'))
df_dim_store = merged_df[['address', 'city', 'country']].copy()
df_dim_store['id_store'] = range(1, len(df_dim_store) + 1)
df_dim_store= df_dim_store[['id_store', 'address', 'city', 'country']]
df_dim_store = df_dim_store.rename(columns={'address': 'address_store', 'city': 'city_store'})

### CAR

In [68]:
cursor = conn.cursor()
try:
    insert_query = """
        INSERT INTO dim_store (id_store, address_store, city_store, country)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (id_store) DO UPDATE
        SET address_store = EXCLUDED.address_store, city_store = EXCLUDED.city_store, country = EXCLUDED.country
    """    
    filas = 0
    for index, row in df_dim_store.iterrows():
        values = tuple(row)
        cursor.execute(insert_query, values)
        filas += 1
    conn.commit()
    print(f"Se afectaron {filas} filas exitosamente.")
except Exception as e:
    conn.rollback()
    print(f"Error durante la inserccion:", e)
finally:
    cursor.close()  

Se afectaron 2 filas exitosamente.
